In [1]:
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


2023-06-24 15:51:23.829268: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-24 15:51:23.912129: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-24 15:51:26.462193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('Input_layer',
                                       min_value=16,
                                       max_value=512,
                                       step=32),
                                       input_shape=(9,)))
    for i in range(hp.Int('num_layers', 2, 50)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=16,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model


In [5]:
def get_model(df,crop_name):
    
    X_tmp = df.iloc[:,:-7]
    y = df[crop_name]
    
    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X_tmp)
    
    tuner = RandomSearch(
        build_model,
        objective='val_mean_absolute_error',
        max_trials=5,
        executions_per_trial=3,
        directory='.',
        project_name=f"keras_tuner_trials/{crop_name}_Model")
   
    #Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    #Search the tuner using the dataset
    tuner.search(X_train, y_train,
                 epochs=100,
                 validation_data=(X_test, y_test))
    
    model = tuner.get_best_models()[0]
    
    model.save(f"Models/{crop_name}")

In [6]:
df = pd.read_csv("Data/training_dataset.csv")
df.set_index("Date", inplace=True)
df.head()

,District Name,ndvi,savi,ndre,Rainfall,Skin_Tmp,Max_Tmp,Min_Tmp,APAR,Potato_Yield,Tomato_Yield,Tomato_b_Yield,Radish_Yield,Carrot_Yield,Cabbage_Yield,Cauliflower_Yield
Date,,,,,,,,,,,,,,,,
2018-12-16,0,0.5155,0.2945,0.3215,0.00,10.87,21.61,8.05,72.62,26.469077,13.831092,70.3,18.254333,18.282961,14.954172,15.410276
2018-12-23,0,0.4030,0.2440,0.2560,0.00,8.52,19.05,6.08,62.54,26.469077,13.831092,70.3,18.254333,18.282961,14.954172,15.410276
2018-12-30,0,0.3730,0.2385,0.2535,0.00,7.86,18.87,7.06,59.80,26.469077,13.831092,70.3,18.254333,18.282961,14.954172,15.410276
2019-01-06,0,0.5010,0.2970,0.3120,2.03,9.43,13.53,8.51,25.16,23.512821,31.328155,78.0,23.997122,13.202756,15.000000,18.314685
2019-01-27,0,0.4180,0.2850,0.2920,0.31,8.12,16.52,4.58,49.37,23.512821,31.328155,78.0,23.997122,13.202756,15.000000,18.314685


In [7]:
df.columns

Index(['District Name', 'ndvi', 'savi', 'ndre', 'Rainfall', 'Skin_Tmp',
       'Max_Tmp', 'Min_Tmp', 'APAR', 'Potato_Yield', 'Tomato_Yield',
       'Tomato_b_Yield', 'Radish_Yield', 'Carrot_Yield', 'Cabbage_Yield',
       'Cauliflower_Yield'],
      dtype='object')

# Tomato model architecture

In [8]:
get_model(df,"Tomato_Yield")

Trial 5 Complete [00h 10m 23s]
val_mean_absolute_error: 6.5654190381368

Best val_mean_absolute_error So Far: 4.449136416117351
Total elapsed time: 00h 37m 19s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Assets written to: Models/Tomato_Yield/assets


INFO:tensorflow:Assets written to: Models/Tomato_Yield/assets


# Potato Model Architecture

In [20]:
get_model(df,"Potato_Yield")

Trial 5 Complete [00h 02m 23s]
val_mean_absolute_error: 1.6732746362686157

Best val_mean_absolute_error So Far: 1.488771677017212
Total elapsed time: 00h 27m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Assets written to: Models/Potato_Yield/assets


INFO:tensorflow:Assets written to: Models/Potato_Yield/assets
